<div class="alert alert-block alert-info" style="text-align:center;">
   <h1 style="font-size: 36px;">ClassVision: AI-Powered Classroom Attendance System</h1>
</div>

**Project Team:**
- `Ankit Kumar Aggarwal`
- `Veerabhadra Rao Marellapudi`
- `Ovadia Sutton`

## Install Required Libraries

In [ ]:
#Install Face Recognition Library
pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=f557233aa8889d42cccbfd8096db654f492506a9d8b0ed67875599b5564a5bbe
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
#Install Retina Face
pip install retina-face

## Import Libraries

In [ ]:
import os
import cv2
import pickle
import random
import face_recognition
import cv2
import pickle
from retinaface import RetinaFace
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

## Function to prepare Embeddings for Known Images

In [ ]:
# Function to load or generate known face encodings and names
def load_or_generate_known_faces():
    # If the pickle file exists, load the known face encodings
    if os.path.exists("known_faces_5050px_a.pickle"):
        with open("known_faces_5050px_a.pickle", "rb") as file:
            known_faces = pickle.load(file)
        return known_faces

    # If the pickle file does not exist, generate the known face encodings and names
    known_faces_encodings = []
    known_face_names = []

    pbar = tqdm(desc="Processing Images")

    # Traverse through each image in the training directory
    training_directory = "/content/drive/MyDrive/AKA/04072024/training"
    for image_name in os.listdir(training_directory):
        if image_name.endswith((".jpg", ".jpeg")):
            image_path = os.path.join(training_directory, image_name)
            pbar.set_postfix({"Current Image": image_name})
            # print(f"Loading image: {image_path}")
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)
            if len(face_encodings) > 0:
                face_encoding = face_encodings[0]
                known_faces_encodings.append(face_encoding)
                known_face_names.append(os.path.splitext(image_name)[0])
            else:
                # print(f"No faces detected in {image_name}. Skipping this image.")
                pass
            pbar.update(1)
    pbar.close()

    # Save the known face encodings and names to a pickle file
    known_faces = (known_faces_encodings, known_face_names)
    with open("known_faces_5050px_a.pickle", "wb") as file:
        pickle.dump(known_faces, file)

    print("Known faces encoding and names saved to 'known_faces_5050px_a.pickle'.")

    return known_faces

In [ ]:
# Load or generate known face encodings and names
known_faces_encodings, known_face_names = load_or_generate_known_faces()

In [ ]:
# Test Images of Classroom with Students
folder_path = "/content/drive/MyDrive/AKA/04152024/test"
image_files = [f for f in os.listdir(folder_path) if f.endswith(".jpg") or f.endswith(".png") or f.endswith(".jpeg")]


In [17]:
# Create a folder for processed images if it doesn't exist
final_folder = "/content/drive/MyDrive/AKA/04152024/Final"
if not os.path.exists(final_folder):
    os.makedirs(final_folder)


## Perform Face Detection and Recognition

In [18]:

# Loop through all images in the folder
for img_file in image_files:
    img_path = os.path.join(folder_path, img_file)
    img = cv2.imread(img_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces using RetinaFace
    face_locations = RetinaFace.detect_faces(img)

    # Convert face_locations to the format expected by face_recognition library
    face_locations = [(face['facial_area'][1], face['facial_area'][2], face['facial_area'][3], face['facial_area'][0]) for face in face_locations.values()]

    # Recognize faces using face_recognition library
    face_encodings = face_recognition.face_encodings(rgb_img, face_locations)

    # Loop through detected faces and their encodings
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare face encodings with known face encodings
        distances = face_recognition.face_distance(known_faces_encodings, face_encoding)

        # Find the index of the closest match
        min_distance_index = distances.argmin()

        # Extract name without "_" and numbers
        name_with_numbers = known_face_names[min_distance_index]
        name = ''.join(filter(str.isalpha, name_with_numbers))

        # Draw a rectangle around the face
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 3)

        # # Add text label with the identity
        # name = known_face_names[min_distance_index]
        # cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Add text label with the identity
        cv2.putText(img, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Save the final image with bounding boxes and labels
    final_img_path = os.path.join(final_folder, img_file)
    cv2.imwrite(final_img_path, img)

print("Final images with bounding boxes and labels saved to 'Final' folder.")


Final images with bounding boxes and labels saved to 'Final' folder.
